In [1]:
import pandas as pd
df = pd.read_csv('decisionaids.csv')
pmids = list(df['pmid'])

In [2]:
from Bio.Entrez import efetch, read

def fetch_abstract(pmid):
    handle = efetch(db='pubmed', id=pmid, retmode='xml')
    xml_data = read(handle)
    xml_data = xml_data['PubmedArticle'][0]
    try:
        article = xml_data['MedlineCitation']['Article']
        if 'Abstract' in article :
            abstract = article['Abstract']['AbstractText'][0]
            return abstract
        else :
            return None
    except IndexError, KeyError:
        return None

In [3]:
abstracts = [fetch_abstract(idx) for idx in pmids]

/home/sarthak/anaconda2/lib/python2.7/site-packages/Bio/Entrez/__init__.py:564: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  E-utilities.""", UserWarning)


In [4]:
df['abstract'] = abstracts

In [5]:
im_map = {'breast cancer': 'BCT', 
          'healthy women at risk of breast cancer': 'BCS', 
          'type II diabetes': 'D',
          'menopausal women': 'MW',
          'pregnant women, previous C section': 'PWC',
          'pregnant women': 'PW',
          'healthy people, at risk colon cancer': 'CCS',
          'prostate cancer': 'PCT',
          'healthy men, contemplating risk of prostate cancer': 'PCS',
          'AF': 'AF',
          'healthy women at genetic risk of breast cancer': 'BCG'}
fdf = df[df['IM_population'].isin(im_map.keys())].reset_index().drop(['index'], axis=1)
fdf = fdf.dropna().reset_index()

In [6]:
fdf.to_csv('decision_aids_filter.csv', encoding='utf-8')